In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC

import sklearn.manifold as skm 
import sklearn.utils.random as skr # for random sampling 


In [3]:
# train data 
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = []
    test_acc = []
    # Early stopping 
    epsilon = 0.001
    counter = 0
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc.append(hdc.fit(traindata, trainlabels, param))
        test_acc.append(hdc.test(testdata, testlabels))
        if len(train_acc) % 5 == 0:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
        if train_acc[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
            break
        if max(test_acc) - test_acc[-1] >= epsilon:
            counter += 1
            #if counter >= 20:
            #    sys.stderr.write("Early stopping initiated")
            #    print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
            #    break
        else:
            counter = 0
    return np.asarray(train_acc), np.asarray(test_acc)

In [4]:
def dump_log(param, train_acc, test_acc, filename):
    joblib.dump((param, train_acc, test_acc), open(filename+".pkl", "wb"), compress=True)
    file = open(filename+".txt", "a")
    msg = str(100*max(train_acc)) + " " + str(100*max(test_acc)) + " " +\
        str(len(train_acc)) + " " + str(np.argmax(test_acc) + 1) + "\n"
    file.write(msg)
    file.close()

In [5]:
############### ISOMAP ########################
# Everyone uses a neighborhood size of 8-12.
# If your manifold is more than 2-3 dimensional, then Isomap probably won't work, 
# you would need to have a neighborhood size that is larger (more points!) so that 
# you have reasonable options to estimate your geodesic path with links to nearby neighbors.
def iso_wrapper(n_neighbors, n_components):
    
    start = time.time()
    isomap = skm.Isomap(n_neighbors, n_components)
    isomap.fit(manifolddata)
    end = time.time()
    # Preparation time 
    prep_time = int(end - start) 
    print( "Prep time: ", prep_time ) 
    
    start = time.time()
    trainiso = isomap.transform(traindata)
    testiso = isomap.transform(testdata)
    end = time.time() 
    # Transformation time
    trans_time = int(end-start)
    print( "Transform time: ", trans_time )

    filename = "./dumper/isomap_data_"+str(n_neighbors)+"_"+str(n_components)+".pkl"
    joblib.dump((trainiso, trainlabels, testiso, testlabels), open(filename, "wb"), compress=True)
    
    return trainiso, testiso, prep_time, trans_time

In [6]:
############### Modified/Hessian Locally Linear Embedding , more ########################

def lle_wrapper(n_neighbors, n_components, lle_type = "modified"):
    
    start = time.time()
    lle = skm.LocallyLinearEmbedding(n_neighbors, n_components, method = lle_type)
    lle.fit(manifolddata)
    end = time.time()
    prep_time = int(end - start) 
    print( prep_time ) 

    start = time.time()
    trainlle = lle.transform(traindata)
    testlle = lle.transform(testdata)
    end = time.time()
    trans_time = int(end-start)
    print( trans_time )

    filename = "./dumper/"+lle_type+"_data_"+str(n_neighbors)+"_"+str(n_components)+".pkl"
    joblib.dump((trainlle, trainlabels, testlle, testlabels), open(filename, "wb"), compress=True)

    return trainlle, testlle, prep_time, trans_time

In [7]:
############### Spectral Embedding ########################
############### Multi-dimensional Scaling ########################
############### t-distributed Stochastic Neighbor Embedding #######
# No online learning version; has to input full data

In [8]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()

Loading dataset MNIST from MNIST
Loading train data... train data of shape (60000, 784) loaded
Loading test data...  test  data of shape (10000, 784) loaded
Data Loaded. Num of features = 784 Num of Classes = 10

In [9]:
# data cropping
traindata = traindata[:20000]
testdata = testdata[:5000]

In [10]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses
print(param)

{'data_location': '../dataset/', 'directory': 'MNIST', 'dataset': 'MNIST', 'D': 2500, 'vector': 'Gaussian', 'mu': 0, 'sigma': 1, 'binarize': 0, 'lr': 0.037, 'sparse': 0, 's': 0.1, 'binaryModel': 0, 'width': None, 'height': None, 'nLayers': 5, 'uniform_dim': 1, 'uniform_ker': 1, 'dArr': None, 'k': 5, 'kArr': None, 'one_shot': 0, 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 'train_percent': 1, 'dropout': 0, 'drop_percentages': [0, 0.1, 0.2, 0.5], 'dropout_rate': 0, 'update_type': <Update_T.FULL: 1>, 'iter_per_trial': 3, 'iter_per_encoding': 5, 'epochs': 250, 'nFeatures': 784, 'nClasses': 10}


In [11]:
# Run manifolding learning data
# Random sampling for manifold 

#manifoldlist = skr.sample_without_replacement(len(traindata), 2000)
#manifolddata = np.asfarray(traindata[np.asarray(manifoldlist)])
#n_neighbors = 200
#n_components  = 25
#param["nFeatures"] = n_components
#traintrans, testtrans, prep_time, trans_time = iso_wrapper(n_neighbors, n_components)

In [12]:
# Run basiline data 
# param["nFeatures"] = nFeatures
# param["manifold"] = "baseline"
# traintrans, testtrans = traindata, testdata

In [13]:
################# VANILLA #################
def vanilla_train(param):
    hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    # Update param with bid
    param = hdb.getParam()
    print(bid)

    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traintrans)
    #trainencoded = hde.encodeData(traindata)
    HDE.saveEncoded(trainencoded, trainlabels, bid, "train")
    testencoded = hde.encodeData(testtrans)
    #testencoded = hde.encodeData(testdata)
    HDE.saveEncoded(testencoded, testlabels, bid, "test")

    train_accs = []
    test_accs = []
    # Should have 95%
    for i in range(param["iter_per_encoding"]):
        hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
        train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
    
    return train_accs, test_accs


In [14]:
# Dimension tuning 
param["D"] = 2500

In [ ]:
for i in range(param["iter_per_trial"]):

    # Run manifolding learning data
    # Random sampling for manifold 

    manifoldlist = skr.sample_without_replacement(len(traindata), 2000)
    manifolddata = np.asfarray(traindata[np.asarray(manifoldlist)])
    n_neighbors = 100
    n_components  = 30
    param["nFeatures"] = n_components
    #param["manifold"] = "isomap"
    param["manifold"] = "modified"
    #traintrans, testtrans, prep_time, trans_time = iso_wrapper(n_neighbors, n_components)
    traintrans, testtrans, prep_time, trans_time = lle_wrapper(n_neighbors, n_components)

    for jj in [100, 200, 500, 1000, 1500, 2000, 2500, 5000]:
        param["D"] = jj
        train_accs, test_accs = vanilla_train(param)
        """
        param["nFeatures"] = nFeatures
        param["manifold"] = "baseline" 
        traintrans, testtrans, prep_time, trans_time = traindata, testdata, 0, 0 
        train_accs, test_accs = vanilla_train(param)
        """

        #filename = "./logfile/" + param["manifold"] + "_" +str(param["D"])+"_"+str(param["id"])
        filename = "./logfile/" + param["manifold"] + "_" +str(param["D"])+"_"+ \
            str(n_components) + "_" + str(n_neighbors) + "_" + str(param["id"])
        file = open(filename+"_time.txt","w")
        file.write("Preparation time: "+str(prep_time)+" Transform time: "+str(trans_time)+"\n")
        file.close()

        for i in range(len(train_accs)):
            dump_log(param, train_accs[i], test_accs[i], filename)


22
218


Generating vanilla HD basis of shape... 


2083


(100, 30)
Encoding time: 1.6007304191589355 
Dumping basis into file: base_2083.pkl 
Encoding data of shape (20000, 30)


Time spent: 11 sec
Dumping data into encoded_2083_train.pkl 
Encoding data of shape (5000, 30)


...

Time spent: 74 sec
Dumping data into encoded_6682_test.pkl 


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.888450 	 	 Test: 0.859800
Train: 0.907600 	 	 Test: 0.876000
Train: 0.916850 	 	 Test: 0.891800
Train: 0.925150 	 	 Test: 0.891600
Train: 0.930950 	 	 Test: 0.894600
Train: 0.934700 	 	 Test: 0.908000
Train: 0.934200 	 	 Test: 0.895600
Train: 0.940500 	 	 Test: 0.922000
Train: 0.940850 	 	 Test: 0.910000
Train: 0.944400 	 	 Test: 0.909600
Train: 0.944500 	 	 Test: 0.921600
Train: 0.946700 	 	 Test: 0.927200
Train: 0.948450 	 	 Test: 0.917600
Train: 0.949650 	 	 Test: 0.927400
Train: 0.951300 	 	 Test: 0.925400
Train: 0.952200 	 	 Test: 0.914600
Train: 0.955050 	 	 Test: 0.921600
Train: 0.956000 	 	 Test: 0.928400
Train: 0.954650 	 	 Test: 0.930000
Train: 0.956450 	 	 Test: 0.935000
Train: 0.956700 	 	 Test: 0.931800
Train: 0.958350 	 	 Test: 0.921000
Train: 0.957450 	 	 Test: 0.927000
Train: 0.959050 	 	 Test: 0.921000
Train: 0.960000 	 	 Test: 0.932200
Train: 0.960250 	 	 Test: 0.924600
Train: 0.963800 	 	 Test: 0.928200
Train: 0.959950 	 	 Test: 0.934400
Train: 0.960150 	 	 

Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.889950 	 	 Test: 0.872000
Train: 0.906400 	 	 Test: 0.886800
Train: 0.915600 	 	 Test: 0.873000
Train: 0.927750 	 	 Test: 0.901600
Train: 0.929650 	 	 Test: 0.908400
Train: 0.932700 	 	 Test: 0.910000
Train: 0.937200 	 	 Test: 0.898200
Train: 0.937550 	 	 Test: 0.916600
Train: 0.941850 	 	 Test: 0.917200
Train: 0.943600 	 	 Test: 0.920800
Train: 0.945000 	 	 Test: 0.927200
Train: 0.949050 	 	 Test: 0.923200
Train: 0.949150 	 	 Test: 0.925800
Train: 0.948650 	 	 Test: 0.918800
Train: 0.951400 	 	 Test: 0.918600
Train: 0.950100 	 	 Test: 0.921200
Train: 0.953400 	 	 Test: 0.929800
Train: 0.953600 	 	 Test: 0.916800
Train: 0.955900 	 	 Test: 0.934400
Train: 0.958850 	 	 Test: 0.921200
Train: 0.955700 	 	 Test: 0.924600
